In [1]:
import requests
from dotenv import load_dotenv
import os 
import pandas as pd
import pygsheets
import json

In [2]:

load_dotenv()

print(os.environ.get('riot_api_key'))

RGAPI-ccb60006-d576-4061-89a9-a9e914dc0067


In [ ]:

service_account = pygsheets.authorize(service_account_file='/Users/usuario/Documentos/riot_project_ds/ML_DataSets_Models/JSONs/vocal-framework-444417-i6-3a85e226df77.json')

sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/18iX-JrmQ8yDy8DINmYUpeVzNgpCVaHjHVwGaO_SL0CI/edit?usp=sharing')

In [4]:
test = sheet.worksheet('title','Test')

In [5]:
test_df = test.get_as_df()

In [6]:
test_df['random_numbers'] = 2

In [7]:
test.set_dataframe(test_df,'A1',copy_head=True)

Riot Connect

In [8]:

api_key = os.environ.get('riot_api_key')
def get_puuid(summonerId=None, gameName=None, tagLine=None, region='americas', api_key=api_key):
    """
    Obtém o PUUID de um jogador usando o summonerId ou gameName/tagLine.
    """
    if api_key is None:
        raise ValueError("API key não fornecida. Certifique-se de que ela está configurada corretamente.")
    
    root_url = f"https://{region}.api.riotgames.com/"
    
    if summonerId:
        # Usando summonerId para buscar o PUUID
        endpoint = 'lol/summoner/v4/summoners/'
        url = f"{root_url}{endpoint}{summonerId}?api_key={api_key}"
        response = requests.get(url)
    elif gameName and tagLine:
        # Usando gameName e tagLine para buscar o PUUID
        endpoint = f"riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}"
        url = f"{root_url}{endpoint}?api_key={api_key}"
        response = requests.get(url)
    else:
        raise ValueError("Você deve fornecer 'summonerId' ou ('gameName' e 'tagLine').")
    
    if response.status_code == 200:
        return response.json().get('puuid')
    else:
        raise Exception(f"Erro na requisição: {response.status_code}, {response.text}")


In [9]:


def get_name_and_tag(puuid = None, api_key=None):
    #link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'
    
    root_url = 'https://americas.api.riotgames.com/'
    endpoint = 'riot/account/v1/accounts/by-puuid/'
    response = requests.get(root_url+endpoint+puuid+'?api_key='+api_key)
    id = {
        'gameName': response.json()['gameName'],
        'tagLine': response.json()['tagLine']
    }
    
    return id


In [10]:
get_puuid(gameName='Arthur Lanches',tagLine='13777',api_key=api_key)

'ccoZVdU65yNzgLSy6lMVC_pTRLs0k_qwJO5zBiCGH1dVAA6hSmbD5z9w8AOhtqsCCHjeMcxH1P0aMg'

In [11]:
get_name_and_tag(puuid=get_puuid(gameName='Arthur Lanches',tagLine='13777',api_key=api_key),api_key=api_key)

{'gameName': 'Arthur Lanches', 'tagLine': '13777'}

In [12]:
def get_ladder():
    root = 'https://br1.api.riotgames.com/'
    chall = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    gm = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    master = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

    chall_response = requests.get(root+chall+'?api_key=' + api_key)
    gm_response = requests.get(root+gm+'?api_key=' + api_key)
    master_response = requests.get(root+master+'?api_key=' + api_key)

    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints',ascending=False).reset_index(drop=True)    
    gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values('leaguePoints',ascending=False).reset_index(drop=True)    
    master_df = pd.DataFrame(master_response.json()['entries']).sort_values('leaguePoints',ascending=False).reset_index(drop=True) 

    ladder = pd.concat([chall_df,gm_df,master_df]).reset_index(drop=True)
    ladder = ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
    ladder['rank']+=1

    return ladder

In [13]:
ladder = get_ladder()

In [14]:
ladder.head()

,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,e0tmU0wNu6aHeCkiKOOXhtESq-QyZkcTpssXkOxx-MPC,1867,125,77,True,False,False,False
1,2,VBS1mQZ6tJ-TBg3a_JXUu84hkKbHHRS-zFIoA-W-o29H-Q...,1702,166,116,True,False,False,False
2,3,eskk-At8RzyeV_rO7n6TpYKf2vT4-g7vTl-_voLA14VUJA,1585,172,130,True,False,False,False
3,4,dBpHOkVP9QDxHH2WnRdYZ-oQ9xN4akFZ33WYlNLlwQNK9w,1412,122,83,True,False,False,True
4,5,3uetW96DdhT0KFrPcataPs0pbV-bpHX0N6uDgiJq5p5P0c...,1375,182,125,True,False,False,False


In [15]:
get_puuid(summonerId='e0tmU0wNu6aHeCkiKOOXhtESq-QyZkcTpssXkOxx-MPC',region='br1')

'ccoZVdU65yNzgLSy6lMVC_pTRLs0k_qwJO5zBiCGH1dVAA6hSmbD5z9w8AOhtqsCCHjeMcxH1P0aMg'

In [39]:
def get_match_history(region = None, puuid = None,start = None, count = 20): 
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}'
    response = requests.get(root_url + endpoint +query_params+ '&api_key='+api_key)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro {response.status_code}: {response.text}")
        return []

In [17]:
def get_match_data_from_id(region = None,matchId = None):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/{matchId}'

    response = requests.get(root_url + endpoint + '?api_key='+api_key)

    return response.json()

In [18]:
get_name_and_tag('f5jJfF_9PDdpLKPrv5psm3uVjj2pw4OpkRDeOYsjJu2Q0UUD93_sFgfxBqKIMww40hogpAPOEIIFzQ',api_key=api_key)

{'gameName': 'Chuck Noyya', 'tagLine': 'BR1'}

In [19]:
get_match_data_from_id(region='americas',matchId='BR1_3066231574')

{'metadata': {'dataVersion': '2',
  'matchId': 'BR1_3066231574',
  'participants': ['7pyulf1m1ptqJAO0R3RF9Ej1NGCtFcSs_rKpJzfDgS71B1ZVinbM_HBoYcux8d5HPHLHMgaQvn8QaA',
   'uDKuD0PIvzCje4wRp8UfB5MRO5Xy0b47Ph7ndeoO1I36HQHKni0s3FYnWkX7OagF-Z8rWY3MYNE1rQ',
   'dEXeKCqgVR7M2elLyvlQ0T95cIrXEGlS98IPymtFNASY5ZHdqF1fxeXfOuZC_7q7kobr-UHVf7NDmw',
   'QsZPihSjagkvjzeOJRdApC9_Ym6esQYME2o27Z2D0WaGvQrx9rh3GDoY1-MvDEg2d3mDbOKO4gR8yw',
   'rwQByoyGEs7xv2N8zXlRYnJF9onUG6AXjdcGxIGJhOldrTLiVXZOodMxCKRqydon_M5PfWTwFwa8Jw',
   'ccoZVdU65yNzgLSy6lMVC_pTRLs0k_qwJO5zBiCGH1dVAA6hSmbD5z9w8AOhtqsCCHjeMcxH1P0aMg',
   'lFMdWgE-J5BoubPqB-ARvxnOQ_z6TlG0Z017JaOpSPlJOmPXhLFq_epqQ8BEzqNiDPNRxY37NxDhgg',
   'eQ9TuNrR1FBUZy925-pv0xmSMht8VK39i5CDgwACx1mlIGPmrkiEQxL0_4emJ9sbhxxs4awiM8bOHA',
   'TetYpA1eoyTQU1uX1lQadQ-5TPqUNdmyZSIGx-Q-FYJoJDJzeinGkqO1W7O_mXb7HcpR-w6QCDxfkQ',
   'Jbt61Bz45MFUa7_9C5c8_cdfiGuVTqJisGWjvKqkJ6hBHzxQtcFEu2Vp-AhJ7_VlkNWWYjOTA9Bm0w']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 1739

In [20]:
match_history = get_match_history(region='americas',puuid='ccoZVdU65yNzgLSy6lMVC_pTRLs0k_qwJO5zBiCGH1dVAA6hSmbD5z9w8AOhtqsCCHjeMcxH1P0aMg',start=250,count = 50)


In [21]:
'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start={start}&count={count}&api_key=RGAPI-5f9f40b7-b7c4-48a8-b213-ed0d928a590e'

'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start={start}&count={count}&api_key=RGAPI-5f9f40b7-b7c4-48a8-b213-ed0d928a590e'

Coletando dados sobre as partidas do Arthur Lanches e montando um DataFrame

In [22]:
partida_teste = get_match_data_from_id(region='americas',matchId='BR1_3066231574')

In [23]:
#420 indica que a partida era Rankeada Solo/duo
partida_teste['info']['queueId']

420

In [ ]:
def get_player_match_data(match_history, target_summoner):
    data = []

    #itera sobre os IDs das partidas
    for matchId in match_history:
        #obtendo os dados da partida com base no matchId
        match = get_match_data_from_id(region='americas', matchId=matchId)
        
        if match['info']['queueId'] == 420:
                #print(match)
                #para cada partida rankeada, pega os dados do participante buscado
                for participant in match['info']['participants']:
                    if participant['riotIdGameName'].strip().lower() == target_summoner.strip().lower():
                        #calculo do kda
                        #print(target_summoner)
                        kda = (participant['kills'] + participant['assists']) / max(1, participant['deaths'])
                        #salvando dados da partida
                        data.append({
                            "matchId": matchId,
                            "queueId": match['info']['queueId'],
                            "gameDuration": match['info']['gameDuration'],
                            "gameStartTimestamp": match['info']['gameStartTimestamp'],
                            "summonerName": participant['summonerName'],
                            "teamId": participant['teamId'],
                            "championName": participant['championName'],
                            "kills": participant['kills'],
                            "deaths": participant['deaths'],
                            "assists": participant['assists'],
                            "kda": round(kda, 2),
                            "totalMinionsKilled": participant['totalMinionsKilled'],
                            "totalDamageTaken":participant["totalDamageTaken"],
                            "totalDamageDealtToChampions": participant['totalDamageDealtToChampions'],
                            "goldEarned": participant['goldEarned'],
                            "visionScore": participant['visionScore'],
                            "summoner1Id": participant['summoner1Id'],
                            "summoner2Id": participant['summoner2Id'],
                            "firstTowerKill":participant["firstTowerKill"],
                            "firstBloodAssist":participant["firstBloodAssist"],
                            "firstBloodKill":participant["firstBloodKill"],
                            "champExperience":participant["champExperience"],
                            "doubleKills":participant["doubleKills"],
                            "tripleKills":participant["tripleKills"],
                            "quadraKills":participant["quadraKills"],
                            "pentaKills":participant["pentaKills"],
                            "win": participant['win']
                        })

    
    df = pd.DataFrame(data)
    return df

In [25]:
df_extract = get_player_match_data(match_history,'Arthur Lanches')

In [26]:
match_history

['BR1_3051797255',
 'BR1_3051779388',
 'BR1_3050961183',
 'BR1_3050940560',
 'BR1_3050920548',
 'BR1_3050898865',
 'BR1_3050873147',
 'BR1_3050851549',
 'BR1_3050840161',
 'BR1_3050822136',
 'BR1_3050478207',
 'BR1_3050457444',
 'BR1_3050368968',
 'BR1_3050361787',
 'BR1_3050280685',
 'BR1_3050271882',
 'BR1_3050266508',
 'BR1_3050080810',
 'BR1_3050063821',
 'BR1_3050051333',
 'BR1_3050036130',
 'BR1_3050009759',
 'BR1_3049984863',
 'BR1_3049971658',
 'BR1_3049678215',
 'BR1_3049659326',
 'BR1_3049636918',
 'BR1_3049619907',
 'BR1_3049606475',
 'BR1_3049590726',
 'BR1_3049575492',
 'BR1_3049559587',
 'BR1_3048942515',
 'BR1_3048827086',
 'BR1_3048754811',
 'BR1_3048725868',
 'BR1_3048702724',
 'BR1_3048689243',
 'BR1_3048658615',
 'BR1_3048632098',
 'BR1_3048279992',
 'BR1_3048258804',
 'BR1_3048235249',
 'BR1_3048214045',
 'BR1_3048200520',
 'BR1_3048176462',
 'BR1_3048161289',
 'BR1_3048146029',
 'BR1_3047819619',
 'BR1_3047799671']

Transformando gameDuration em minutos e GameStart na data que foi jogada a partida

In [27]:
df_extract["gameDurationMin"] = (df_extract["gameDuration"] / 60).round(1)

In [28]:
df_extract["gameStartTime"] = pd.to_datetime(df_extract["gameStartTimestamp"], unit="ms")
df_extract["gameStartTimeFormatted"] = df_extract["gameStartTime"].dt.strftime('%Y-%m-%d %H:%M:%S')


In [29]:
df_extract

,matchId,queueId,gameDuration,gameStartTimestamp,summonerName,teamId,championName,kills,deaths,assists,...,firstBloodKill,champExperience,doubleKills,tripleKills,quadraKills,pentaKills,win,gameDurationMin,gameStartTime,gameStartTimeFormatted
0,BR1_3051797255,420,1981,1736617589784,Arthur Lanches,100,Teemo,6,4,10,...,False,18441,0,0,0,0,True,33.0,2025-01-11 17:46:29.784,2025-01-11 17:46:29
1,BR1_3051779388,420,1682,1736615516086,Arthur Lanches,100,Teemo,10,1,5,...,False,17082,1,0,0,0,True,28.0,2025-01-11 17:11:56.086,2025-01-11 17:11:56
2,BR1_3050961183,420,1650,1736460584762,Arthur Lanches,100,Teemo,7,6,3,...,False,13193,1,0,0,0,False,27.5,2025-01-09 22:09:44.762,2025-01-09 22:09:44
3,BR1_3050940560,420,1551,1736458644616,Arthur Lanches,200,Teemo,5,2,5,...,False,14232,1,0,0,0,True,25.8,2025-01-09 21:37:24.616,2025-01-09 21:37:24
4,BR1_3050920548,420,1897,1736456300963,Arthur Lanches,200,Teemo,9,4,7,...,False,17014,1,0,0,0,True,31.6,2025-01-09 20:58:20.963,2025-01-09 20:58:20
5,BR1_3050898865,420,1553,1736453862180,Arthur Lanches,200,Teemo,6,4,5,...,False,12525,0,0,0,0,False,25.9,2025-01-09 20:17:42.180,2025-01-09 20:17:42
6,BR1_3050873147,420,2424,1736449792210,Arthur Lanches,100,Varus,12,6,4,...,False,20557,2,0,0,0,True,40.4,2025-01-09 19:09:52.210,2025-01-09 19:09:52
7,BR1_3050851549,420,2260,1736447053714,Arthur Lanches,100,Teemo,7,3,9,...,False,21456,0,0,0,0,False,37.7,2025-01-09 18:24:13.714,2025-01-09 18:24:13
8,BR1_3050840161,420,956,1736445289063,Arthur Lanches,200,Teemo,5,0,3,...,True,9000,0,0,0,0,True,15.9,2025-01-09 17:54:49.063,2025-01-09 17:54:49
9,BR1_3050822136,420,2021,1736442506430,Arthur Lanches,200,Teemo,6,6,5,...,False,19600,1,0,0,0,True,33.7,2025-01-09 17:08:26.430,2025-01-09 17:08:26


In [ ]:
#df_extract.to_csv('/Users/usuario/Documentos/riot_project_ds/ML_DataSets_Models/Csv/ArthurLanches250-300.csv',index = False)

Salvei todos os dados a partir do inicio da season 2025 no dia 09/01/2025
até o presente momento 2025-02-13, esses serão os dados utilizados, no atual momento o arthur lanches
é o top1 do servidor

Funcao para recolher o CSV de determinado player de uma unica vez, recebe o numero de partidas que deve ser coletado e o nick do jogador, junto ao puuid

In [ ]:
def save_matches_df(number_matches,nick,puuid): 
    i = 0
    for n in range(i,number_matches,10):
        match_history = get_match_history(region='americas',puuid=puuid,start=n,count = 50)
        df_extract = get_player_match_data(match_history,nick)
        df_extract["gameDurationMin"] = (df_extract["gameDuration"] / 60).round(1)
        df_extract["gameStartTime"] = pd.to_datetime(df_extract["gameStartTimestamp"], unit="ms")
        df_extract["gameStartTimeFormatted"] = df_extract["gameStartTime"].dt.strftime('%Y-%m-%d %H:%M:%S')
        df_extract.to_csv(f'/Users/usuario/Documentos/riot_project_ds/ML_DataSets_Models/Csv/{nick}-{str(n)}-{number_matches}.csv', index=False)

In [47]:
nick = 'Mobyt'
tag = 'ZOE'
puuid_player = get_puuid(gameName = nick,tagLine = tag,api_key=api_key)
save_matches_df(10,nick,puuid_player)

['BR1_3065860563', 'BR1_3065852473', 'BR1_3065836314', 'BR1_3065829192', 'BR1_3065823433', 'BR1_3065311868', 'BR1_3065295100', 'BR1_3064764299', 'BR1_3064694552', 'BR1_3064687297', 'BR1_3064677750', 'BR1_3064672710', 'BR1_3064670026', 'BR1_3064540954', 'BR1_3064521712', 'BR1_3062033085', 'BR1_3054243541', 'BR1_3054229940', 'BR1_3054224696', 'BR1_3053866322', 'BR1_3053840810', 'BR1_3053822296', 'BR1_3053800192', 'BR1_3051920232', 'BR1_3051914682', 'BR1_3051902498', 'BR1_3051888373', 'BR1_3051582795', 'BR1_3051564014', 'BR1_3051545312', 'BR1_3051510397', 'BR1_3051488759', 'BR1_3051465320', 'BR1_3051442548', 'BR1_3051420224', 'BR1_3051400032', 'BR1_3051381504', 'BR1_3051357110', 'BR1_3051342652', 'BR1_3051335198', 'BR1_3051312873', 'BR1_3051300991', 'BR1_3051292191', 'BR1_3050752864', 'BR1_3046765914', 'BR1_3046744843', 'BR1_3046720170', 'BR1_3046702721', 'BR1_3046568150', 'BR1_3046256764']
{'metadata': {'dataVersion': '2', 'matchId': 'BR1_3065860563', 'participants': ['3Fc2dewecjjND1DuZR